# Shortest Path revisited and NP complete problems

## Single source shortest path

- directed graph $G = (V, E)$
- for every destonation $v$ and source $s$ in $V$, compute the length of the shortest s-v path

Dijkstra
- $O(mlogn)$ with heaps
- $m$ = number of edges
- $n$ = number of nodes
- edges must have positive cost

Bellman-Ford
- either compute the length of the shortest s-v path or a negative cycle
- for every $v$ in $V$ and $i = 1 \dots n$, let $P$ = shortest s-v path with at most $i$ edges
- case 1: if $P$ has $\le (i-1)$ edges, it is a shortest s-v path with $\le (i-1)$ edges
- case 2: if $P$ has $i$ edges with the last hop $(w,v)$m then $P^{'}$ is a shortest s-v path with $\le (i-1)$ edges

Recurrence
- let $L_{i,v}$ = minimum length of a s-v path with $\le i$ edges (with cycles allowed)
- for every $v$ in $V$, $i = 1 \dots n$

$L_{i,v} = min\left[L_{i-1,v}, min\left[L_{i-1,v},c_{wv}\right]\right]$

- If no negative cycles
    - shortest path do not have cycles
    - have $\le (n-1)$ edges
    - compute $L_{i,v}$ for all $i = 1 \dots n-1$ and all $v$ in $V$
    
Pseudo code
- let $A$ = 2-D array (index $i$ and $v$)
- base case: $A[0,s] = 0$ and $A[0,v] = +\infty$ for all $v != s$
- for i = 1 to n-1
    - for each $v$ in $V$
        - $A[i,v] = min\left[A_{i-1,v}, min\left[A_{i-1,v},c_{wv}\right]\right]$
- if $G$ has no negative cycle, then answer is $A[n-1,v]$

Stopping early
- suppose for some $j < n-1$, $A[j,v] = A[j-1,v]$ for all vertices $v$
- for all $v$, all future $A[i,v]'s will be the same$
- can safely halt

Negative cycle detection
- $G$ has no negative cycle iff $A[n-1,v] = A[n,v]$ for all $v$ in $V$
- can perform one extra interation where $i = n$

Space optimization
- only need $A[i-1,v]$'s to compute $A[i,v]$'s
- thus, only need $O(n)$ to remember current and last rounds of subproblems
- compute a second table B where $B[i,v]$ = 2nd-to-last vertex on a shortest s-v path with $\le i $ edges (or NULL if no such path exists)
- assume input graph $G$ has no negative cycles and we correctly compute $B[i,v]$'s
- then, tracing back predecessor pointers $B[n-1,v]$'s from $v$ to $s$ yields a shortest s-v path
- base case: $B[0,v]$ = NULL for all $v$ in $V$
- to compute $B[i,v]$ with $i > 0$
- case 1: $B[i,v] = B[i-1,v]$
- case 2: $B[i,v]$ = the vertex $x$, $w$ achieving the minimum

## All pairs shortest path

- no distinguished source vertex
- shortest s-v path for all pairs of vertices $u,v$ in $V$ or report that $G$ contains a negative cycle

Floyd-Warshall
- order vertices $V = \{1 \dots n\}$ arbitrarily
- let $V^{(k)} = \{1 \dots k\}$
- suppose $G$ has no negative cycle
- fix source $i$ in $V$, destination $j$ in $V$, and $k$ in $\{1 \dots k\}$ 
- let $p$ = shortest (cycle free) i-j path with all internal nodes in $V^{(k)}$

Then
- case 1: if $k$ is not internal to $p$, the $p$ is a shortest (cycle free) i-j path with all internal nodes in $V^{(k-1)}$ 
- case 2: if $k$ is internal to $p$, then
    - $p_{1}$ = shortest (cycle free) i-k path with all internal nodes in $V^{(k-1)}$
    - $p_{2}$ = shortest (cycle free) k-j path with all internal nodes in $V^{(k-1)}$
    
Pseudo code
- let $A$ = 3-D array (nidex i,j,k)
- $A[i,j,k]$ = length of a shortest i-j path with all internal nodes in $\{1 \dots k\}$
- base case: for all $i,j$ in $V$
    - $A[i,j,0]$ = 0 if $i$ = $j$, $C_{(ij)}$ if $(i,j)$ in $E$, $+\infty$ if $i$ != $j$ and $(i,j)$ not in $E$
- for k =1 to n
    - for i = 1 to n
        - for j = 1 to n
            $A[i,j,k]$ = $min\left[A[i,j,k-1], A[i,k,k-1] + A[k,j,k-1]\right]$
- negative cycle? 
    - will have $A[i,i,n] < 0$ for at least one $i$ in $V$ at the end of algorithm
- reconstruct a shortest i-j path? 
    - in addition to $A$, have Floyd-Warshall compute $B[i,j]$ = max level of an internal node on a shortest i-j path for all $i,j$ in $V$ 
    - reset $B[i,j] = k$ if 2nd case of recurrence used to compute $A[i,j,k]$
    - can use $B[i,j]$'s to recursively reconstruct shortest paths
    
Johnson's algorithm
- reduce to all pairs shortest path
    - n Dijkstra
    - 1 Bellman-Ford
- reweighting using vertex weights $\{p_{v}\}$ adds the same amount (namely, $p_{s} - p_{t}$) to every s-t path
- reweighting always leaves the shortest path unchanged

Example
- define vertex weight $p_{v}$ = length of shortest s-v path
- for every edge $e=(u,v)$, define $c^{'}_{e}$ = $c_{e}$ + $p_{u}$ - $p_{v}$
- after reweighting, all edge length are non-negative

In summary
1. form $G^{'}$ by adding a new vertex $s$ and a new edge $(s,v)$ with length 0 for each $v$ in $G$
2. run Bellman-Ford on $G^{'}$ with source vertex s
3. for each $v$ in $G$, define $p_{v}$ = length of a shortest s-v path in $G^{'}$. For each edge $e=(u,v)$ in $G$, define $c^{'}_{e}$ = $c_{e}$ + $p_{u}$ - $p_{v}$
4. for each vertex $u$ of $G$, run Dijkstra in $G$, with edge lengths $\{c^{'}_{e}\}$, with source vertex $u$, to compute the shortest path distance $d^{'}(u,v)$ for each $v$ in $G$
5. for each pair $u,v$ in $G$, return the shortest path distance $d(u,v)$ = $d^{'}(u,v)$ - $p_{u}$ + $p_{v}$

## NP complete

- polynomial-time solvable? $O(n^{k})$ time
- P = a set of polynomial-time solvable problems
- let $C$ = a set of problems. Problem $\pi$
    - $\pi$ in $C$
    - everything in $C$ reduces to $\pi$
- that is, $\pi$ is the hardest problem in all of $C$
- a problem is in NP if
    - solutions always have length polynomial in the input size
    - solutions can be verified in polynomial time
- every problem in NP can be solved by brute-force search in exponential time
- a polynomial-time algorithm for one NP-complete problem solves every problem in NP efficiently [P = NP]
- but generally, P != NP
- NP-completeness of $\pi$
    - find a known NP-complte problem $\pi^{'}$
    - prove that $\pi^{'}$ reduced to $\pi$

Vertex cover problem
- given an undirected graph $G=(V,E)$
- compute a minimum-cardinality subset $S$ in $V$ that contains at least one endpoint of each edge of $G$
- given a positive integer $k$ as input, we want to check whether or not there is a vertex cover with size $\le k$
- consider graph $G$, edge $(u,v)$ in $G$, integer $k \ge 1$
- let $G_{u} = G$ with $u$ and its incident edged deleted
- let $G_{v} = G$ with $v$ and its incident edged deleted
- then, $G$ has a vertex cover of size $k$ iff $G_{u}$ or $G_{v}$ or both have a vertex cover of size $(k-1)$

Recurrence
- ignore base case
- pick an arbitrary edge $(u,v)$ in $E
- recursively search for a vertex cover $S$ of size $(k-1)$ in $G_{u}$. If found, return $S$ plus $u$
- recursively search for a vertex cover $S$ of size $(k-1)$ in $G_{v}$. If found, return $S$ plus $v$


Traveling salesman problem
- given undirected graph with non-negative edge cost, find min cost to visit all vertices
- to enforce constraint that each vertex visited exactly once, need to remember the "identities" of vertices visited in a sub-problem
- sub-problem: for every destination $j = \{1 \dots n \}$, every subset $S$ in $\{1 \dots n \}$ that contains $1$ and $j$, let $L_{s,j}$ = minimum length of a path from $1$ to $j$ that visits precisely the vertices of $S$ (exactly once each)
